
# Задание 3.1 - Сверточные нейронные сети (Convolutional Neural Networks)

Это последнее задание на numpy, вы до него дожили! Остался последний марш-бросок, дальше только PyTorch.  
В этом задании вы реализуете свою собственную сверточную нейронную сеть.

In [1]:
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline

%load_ext autoreload
%autoreload 2

In [2]:
from dataset import load_svhn, random_split_train_val
from gradient_check import check_layer_gradient, check_layer_param_gradient, check_model_gradient
from layers import FullyConnectedLayer, ReLULayer, ConvolutionalLayer, MaxPoolingLayer, Flattener
from model import ConvNet
from trainer import Trainer, Dataset
from optim import SGD, MomentumSGD
from metrics import multiclass_accuracy

### Загружаем данные

На этот раз мы не будем их преобразовывать в один вектор, а оставим размерности `(num_samples, 32, 32, 3)`.

In [3]:
def prepare_for_neural_network(train_X, test_X):    
    train_X = train_X.astype(float) / 255.0
    test_X = test_X.astype(float) / 255.0
    
    # Subtract mean
    mean_image = np.mean(train_X, axis = 0)
    train_X -= mean_image
    test_X -= mean_image
    
    return train_X, test_X
    
train_X, train_y, test_X, test_y = load_svhn("data", max_train=10000, max_test=1000)    
train_X, test_X = prepare_for_neural_network(train_X, test_X)
# Split train into train and val
train_X, train_y, val_X, val_y = random_split_train_val(train_X, train_y, num_val = 1000)

### Реализуем новые слои!
Сначала основной новый слой - сверточный (Convolutional layer). Для начала мы реализуем его для только одного канала, а потом для нескольких.

Сверточный слой выполняет операцию свертки (convolution) с весами для каждого канала, а потом складывает результаты. Возможно, поможет пересмотреть Лекцию 6 или внимательно прочитать http://cs231n.github.io/convolutional-networks/

Один из подходов к реализации сверточного слоя основан на том, что для конкретного "пикселя" выхода применение сверточного слоя эквивалентно обычному полносвязному.

![Getting Started](conv_img.jpg)

Рассмотрим один такой "пиксель":

Он получает на вход  
регион входа I размера `(batch_size, filter_size, filter_size, input_channels)`,

применяет к нему веса W `(filter_size, filter_size, input_channels, output_channels` и выдает `(batch_size, output_channels)`.

Если:  
* вход преобразовать в I' `(batch_size, filter_size*filter_size*input_channels)`,  
* веса в W' `(filter_size*filter_size*input_channels, output_channels)`,

то выход "пикселе" будет эквивалентен полносвязному слою со входом I' и весами W'.  
Осталось выполнить его в цикле для каждого пикселя :)

In [4]:
# TODO: Implement ConvolutionaLayer that supports only 1 output and input channel

# Note: now you're working with images, so X is 4-dimensional tensor of
# (batch_size, height, width, channels)

np.random.seed(0)
X = np.array([
              [
               [[1.0], [2.0]],
               [[0.0], [-1.0]]
              ]
              ,
              [
               [[0.0], [1.0]],
               [[-2.0], [-1.0]]
              ]
             ])

# Batch of 2 images of dimensions 2x2 with a single channel
print("Shape of X:",X.shape)

layer = ConvolutionalLayer(in_channels=1, out_channels=1, filter_size=2, padding=0)
print("Shape of W", layer.W.value.shape)
layer.W.value = np.zeros_like(layer.W.value)
layer.W.value[0, 0, 0, 0] = 1.0
layer.B.value = np.ones_like(layer.B.value)
result = layer.forward(X)

assert result.shape == (2, 1, 1, 1)
assert np.all(result == X[:, :1, :1, :1] +1), "result: %s, X: %s" % (result, X[:, :1, :1, :1])


# Now let's implement multiple output channels
layer = ConvolutionalLayer(in_channels=1, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
assert result.shape == (2, 1, 1, 2)


# And now multple input channels!
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0]],
               [[0.0, -1.0], [-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [1.0, -1.0]],
               [[-2.0, 2.0], [-1.0, 0.0]]
              ]
             ])

print("Shape of X:", X.shape)
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
assert result.shape == (2, 1, 1, 2)

Shape of X: (2, 2, 2, 1)
Shape of W (2, 2, 1, 1)
Shape of X: (2, 2, 2, 2)


### А теперь имплементируем обратный проход

Возможно, это самое сложное место в курсе. Дальше будет лучше.

Раз выполнение сверточного слоя эквивалентно полносвязному слою для каждого "пикселя" выхода, то общий обратный проход эквивалентен обратному проходу каждого из таких "слоев".  
Градиенты от каждого из этих "слоев" в каждом пикселе надо сложить в соответствующие пиксели градиента по входу, а градиенты весов сложить все вместе.

In [14]:
# First test - check the shape is right
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
result = layer.forward(X)
d_input = layer.backward(np.ones_like(result))
assert d_input.shape == X.shape

# # Actually test the backward pass
# # As usual, you'll need to copy gradient check code from the previous assignment
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_gradient(layer, X)

layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_param_gradient(layer, X, 'W')
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=2, padding=0)
assert check_layer_param_gradient(layer, X, 'B')

self.W.grad
  [[[[ 1.  1.]
   [ 1.  1.]]

  [[ 3.  3.]
   [ 0.  0.]]]


 [[[-2. -2.]
   [ 1.  1.]]

  [[-2. -2.]
   [-2. -2.]]]]
CHECK GRADIENT
x is 
 [[[[ 1.  0.]
   [ 2.  1.]]

  [[ 0. -1.]
   [-1. -2.]]]


 [[[ 0.  1.]
   [ 1. -1.]]

  [[-2.  2.]
   [-1.  0.]]]]
self.W.grad
  [[[[ 0.83265074  0.32706621]
   [ 1.63159743  0.37775917]]

  [[ 3.29689891  1.03189159]
   [-0.79894669 -0.05069296]]]


 [[[-3.26319486 -0.75551834]
   [ 2.43054412  0.42845213]]

  [[-2.46424817 -0.70482538]
   [-1.66530148 -0.65413242]]]]
analytic grad is 
 [[[[ 0.95391504 -0.52841742]
   [ 0.34578707  0.65632938]]

  [[ 0.14372389  0.86567082]
   [ 0.08690502  1.00134383]]]


 [[[ 1.91996079 -0.63263871]
   [ 0.53784188  0.91825745]]

  [[ 0.20295892  2.10416283]
   [-0.20497892  1.78365021]]]]
self.W.grad
  [[[[ 0.83265907  0.32706948]
   [ 1.63159743  0.37775917]]

  [[ 3.29689891  1.03189159]
   [-0.79894669 -0.05069296]]]


 [[[-3.26319486 -0.75551834]
   [ 2.43054412  0.42845213]]

  [[-2.46424817 -0.


Осталось реализовать дополнение нулями (padding).  
Достаточно дополнить входной тензор нулями по сторонам. Не забудьте учесть это при обратном проходе!

In [6]:
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0]],
               [[0.0, -1.0], [-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [1.0, -1.0]],
               [[-2.0, 2.0], [-1.0, 0.0]]
              ]
             ])
print("X shape ", X.shape)            
layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=3, padding=1)
result = layer.forward(X)
print("result shape is ", result.shape)
# Note this kind of layer produces the same dimensions as input
assert result.shape == X.shape,"Result shape: %s - Expected shape %s" % (result.shape, X.shape)
d_input = layer.backward(np.ones_like(result))
assert d_input.shape == X.shape
# layer = ConvolutionalLayer(in_channels=2, out_channels=2, filter_size=3, padding=1)
# assert check_layer_gradient(layer, X)

X shape  (2, 2, 2, 2)
result shape is  (2, 2, 2, 2)



### После следующего слоя вам уже будет все ни по чем - max pooling

Max Pooling - это слой, реализующий операцию максимума для каждого канала отдельно в окресности из pool_size "пикселей".

![Getting Started](max_pooling.jpg)

In [70]:
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0]],
               [[0.0, -1.0], [-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [1.0, -1.0]],
               [[-2.0, 2.0], [-1.0, 0.0]]
              ]
             ])
print("X shape ", X.shape) 


pool = MaxPoolingLayer(2, 2)
result = pool.forward(X)
print("result is\n", result)
assert result.shape == (2, 1, 1, 2)

assert check_layer_gradient(pool, X)

X shape  (2, 2, 2, 2)
result is
 [[[[2. 1.]]]


 [[[1. 2.]]]]
CHECK GRADIENT
x is 
 [[[[ 1.  0.]
   [ 2.  1.]]

  [[ 0. -1.]
   [-1. -2.]]]


 [[[ 0.  1.]
   [ 1. -1.]]

  [[-2.  2.]
   [-1.  0.]]]]
analytic grad is 
 [[[[ 0.          0.        ]
   [-0.30374029 -0.10540882]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.52376836  0.        ]]

  [[ 0.          0.14297433]
   [ 0.          0.        ]]]]
numeric grad array is 
 [[[[ 0.          0.        ]
   [-0.30374029 -0.10540882]]

  [[ 0.          0.        ]
   [ 0.          0.        ]]]


 [[[ 0.          0.        ]
   [ 0.52376836  0.        ]]

  [[ 0.          0.14297433]
   [ 0.          0.        ]]]]
Gradient check passed!


In [83]:
X = np.array([
              [
               [[1.0, 0.0], [2.0, 1.0], [-1.0, -2.0], [-1.0, -2.0]],
               [[0.0, -1.0], [-1.0, -2.0], [-1.0, -2.0],[-1.0, -2.0]],
               [[-2.0, 2.0], [-1.0, -2.0], [-1.0, 0.0],[-1.0, -2.0]],
               [[-2.0, 2.0], [-1.0, -2.0], [-1.0, 0.0],[-1.0, -2.0]]
              ]
              ,
              [
               [[0.0, 1.0], [-1.0, -2.0], [1.0, -1.0],[-1.0, -2.0]],
               [[-2.0, 2.0], [-1.0, -2.0], [-1.0, 0.0], [-1.0, -2.0]],
               [[-2.0, 2.0], [-1.0, -2.0], [-1.0, 0.0], [-1.0, -2.0]],
               [[-2.0, 2.0], [-1.0, -2.0], [-1.0, 0.0],[-1.0, -2.0]]
              ]
             ])
print("X shape ", X.shape) 
print("X[0]\n", X[:,:,:,0])
# print("X[1]\n", X[:,:,:,1])


pool = MaxPoolingLayer(2, 2)
result = pool.forward(X)
print("result is\n", result)
d_input = pool.backward(result)
# assert result.shape == (2, 1, 1, 2)

# assert check_layer_gradient(pool, X)

X shape  (2, 4, 4, 2)
X[0]
 [[[ 1.  2. -1. -1.]
  [ 0. -1. -1. -1.]
  [-2. -1. -1. -1.]
  [-2. -1. -1. -1.]]

 [[ 0. -1.  1. -1.]
  [-2. -1. -1. -1.]
  [-2. -1. -1. -1.]
  [-2. -1. -1. -1.]]]
result is
 [[[[ 2.  1.]
   [-1. -2.]]

  [[-1.  2.]
   [-1.  0.]]]


 [[[ 0.  2.]
   [ 1.  0.]]

  [[-1.  2.]
   [-1.  0.]]]]
fragment x 
 [[ 1.  2.]
 [ 0. -1.]]
ind is 
 [0 1]
fragment x 
 [[-1. -1.]
 [-1. -1.]]
ind is 
 [0 0]
fragment x 
 [[-2. -1.]
 [-2. -1.]]
ind is 
 [0 1]
fragment x 
 [[-1. -1.]
 [-1. -1.]]
ind is 
 [0 0]
fragment x 
 [[ 0.  1.]
 [-1. -2.]]
ind is 
 [0 1]
fragment x 
 [[-2. -2.]
 [-2. -2.]]
ind is 
 [0 0]
fragment x 
 [[ 2. -2.]
 [ 2. -2.]]
ind is 
 [0 0]
fragment x 
 [[ 0. -2.]
 [ 0. -2.]]
ind is 
 [0 0]
fragment x 
 [[ 0. -1.]
 [-2. -1.]]
ind is 
 [0 0]
fragment x 
 [[ 1. -1.]
 [-1. -1.]]
ind is 
 [0 0]
fragment x 
 [[-2. -1.]
 [-2. -1.]]
ind is 
 [0 1]
fragment x 
 [[-1. -1.]
 [-1. -1.]]
ind is 
 [0 0]
fragment x 
 [[ 1. -2.]
 [ 2. -2.]]
ind is 
 [1 0]
fragment x 
 [[-1. 

И на закуску - слой, преобразующий четырехмерные тензоры в двумерные.

Этот слой понадобится нам, чтобы в конце сети перейти от сверточных слоев к полносвязным.

In [84]:
flattener = Flattener()
result = flattener.forward(X)
assert result.shape == (2,8)

assert check_layer_gradient(flattener, X)

AssertionError: 

### Теперь есть все кирпичики, создаем модель

In [87]:
# TODO: In model.py, implement missed functions function for ConvNet model

# No need to use L2 regularization
model = ConvNet(input_shape=(32,32,3), n_output_classes=10, conv1_channels=2, conv2_channels=2)
loss = model.compute_loss_and_gradients(train_X[:2], train_y[:2])

# TODO Now implement backward pass and aggregate all of the params
check_model_gradient(model, train_X[:2], train_y[:2])

Checking gradient for W1
CHECK GRADIENT
x is 
 [[[[ 0.55727908  1.54882947]
   [-1.70367018  0.14685858]
   [ 1.57739465  0.77912164]]

  [[ 2.16594136 -0.49624807]
   [ 1.08521047  0.71844019]
   [ 0.05058829 -1.01231956]]

  [[ 0.22373236  1.9337601 ]
   [ 1.24988081  0.48841608]
   [-0.49391724 -1.40751252]]]


 [[[-0.19902985 -0.90898711]
   [-0.35590979  1.10262794]
   [ 1.22524106  1.08717756]]

  [[ 0.54415423 -0.52979635]
   [ 0.43899802 -1.72670806]
   [ 1.10404606 -1.09934292]]

  [[ 1.33413655 -0.63354464]
   [ 2.23311422 -0.26385797]
   [-0.1156972   0.76240023]]]


 [[[-1.34186612  0.21859976]
   [-1.00528307  0.67685075]
   [ 0.49975637  0.2243302 ]]

  [[ 1.23618233  0.14349703]
   [-2.19476646 -0.7532163 ]
   [-0.95958156  0.66338589]]

  [[-0.20235996 -0.64302906]
   [-0.81345253 -0.34161743]
   [ 0.42184006  0.88950668]]]]
analytic grad is 
 [[[[-5.65171082e-04 -3.97847271e-04]
   [-5.09973165e-04 -8.05150719e-04]
   [-3.88429285e-04 -1.63802219e-03]]

  [[-1.21961138

True

In [94]:
X = np.array([
              [
               [[1.0, 1, 0.0], [2.0, 1, 1.0], [9.0, 1, 1.0]],
               [[0.0, 1,  -1.0], [-1.0, 1,  -2.0],[2.0, 1, 1.0]], 
               [[0.0, 1,  -1.0], [-5.0, 1,  -2.0],[2.0, 9, 1.0]],
               [[0.0, 1,  3.0], [13.0, 1,  -2.0],[2.0, 1, 1.0]]
              ]
              ,
              [
               [[0.0, 1, 1.0], [1.0, 1,  -1.0], [2.0, 1, 1.0]],
               [[-2.0, 1,  2.0], [-1.0, 1, 0.0], [2.0, 1, 1.0]],
               [[-2.0, 1,  2.0], [-1.0, 1, 0.0], [2.0, 1, 1.0]],
               [[0.0, 1,  -1.0], [-1.0, 22,  -2.0],[2.0, 1, 1.0]]
              ]
             ])

# A = np.array([[[3,3], [2,2]], [[3,3,], [1,1]]])
x_off= 1
y_off = 1
Y = X[:, x_off:2+x_off,  y_off:2 + y_off, :]
print("X shape is ", X.shape)
# print("Y shape is ", Y.shape)
# print(X)
# print(X.ndim)
# print("Y is\n", Y)
# print("A\n", A)
print("transformed X shape is ", X.T.shape)

b = [1000, 100, 30]
# print(X + b)
maxes = X.max(axis=(1, 2))
# indices = X.argmax(axis=(1, 2))
# indices = X.argmax(axis= (1,2))
x, y = np.argwhere(maxes==maxes.max())[0]
print(maxes)
print(x, y)
# print(maxes[ind[0], ind[1]])
# print(maxes.shape)


X shape is  (2, 4, 3, 3)
transformed X shape is  (3, 3, 4, 2)
[[13.  9.  3.]
 [ 2. 22.  2.]]
1 1
